<h3>General Imports</h3>

In [ ]:
import os
import sys
import time

# Mantid imports
from mantid.simpleapi import LoadEventNexus

# drtsans imports
from drtsans.pixel_calibration import calculate_barscan_calibration, load_calibration, as_intensities

In [ ]:
# "plot_wing_detector" is a utility plotting function that we will use a couple of times
%matplotlib notebook
from drtsans.plots import plot_detector
def plot_wing_detector(input_workspace, axes_mode='tube-pixel', panel_name='wing_detector'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})

<h3>Dataset</h3>

The barscan are a group of files for BIOSANS wing detector.

We assume formula $y = dcal - 565$ to translate the position of the bar ($dcal$) stored in the metadata of the runs, and the position of the bar in the frame of reference of the sample ($y$).

In [ ]:
data_dir = '/HFIR/CG3/IPTS-23782/nexus/'
first_run, last_run = 838, 953
detector_array = 'wing_detector'  # calibration for the wing detector
formula = '{y} - 565'  # translate from scan log value to Y-coordinate in the sample's reference frame.
# Gather all file paths into a python list
barscan_files = [os.path.join(data_dir, f'CG3_{run}.nxs.h5') for run in range(first_run, 1 + last_run)]

<h3>Caveats</h3>
As it turns out, the bar is not covering the last tube. We verify this by plotting the intensities on the wing detector for the middle scan.

We have created a **mask file** whereby we mask the last tube. This flags this tube as faulty when performing the barscan calculations. **Average** pixel positions and heights will be used to estimate the calibration of this faulty tube.

In [ ]:
middle_scan_index = int((last_run - first_run) / 2)
LoadEventNexus(barscan_files[middle_scan_index], OutputWorkspace='middle_scan')
plot_wing_detector('middle_scan')
mask_file = '/home/jbq/repositories/sans-backend/sans-backend2/notebooks/barscan/biosans_mask_bank88_tube4.xml'

<h3>Calibration</h3>
We carry out the calibration. There are 116 files but we use one third of them (see statement <code>barscan_files[::3]</code> in the following cell).

Calculation of the calibration is slow, about 5 to 10 minutes. However, calibrations are carried out once or twice in a cycle.

In [ ]:
# Carry out the calibration
start_time = time.time()
calibration = calculate_barscan_calibration(barscan_files[::3], component=detector_array,
                                            formula=formula, mask=mask_file)
print('Calibration took ', int((time.time() - start_time) / 60), 'minutes')

<h3>Saving the Calibration</h3>
There are default database files for each instrument when saving a calibration. For BIOSANS is <code>/HFIR/CG3/shared/calibration/pixel_calibration.json</code>. We don't want to mess with the official database so we save this calibration to a temporary database file.
We use argument overwrite=True in case we run the notebook more than once. Then we will overwrite the existing 
calibration entry in the database

In [ ]:
calibration.save(database='/HFIR/CG3/shared/tmp/calibration.json', overwrite=True)

<h3>Applying a Saved Calibration</h3>
Next we use one of the barscan files as the target for our recently saved calibration.
Notice that we are applying the calibration to our input workspace and saving the result to an output workspace. If you want to <b>overwrite</b> the input workspace, then omit the <code>output_workspace</code> argument.

In [ ]:
LoadEventNexus(os.path.join(data_dir, f'CG3_895.nxs.h5'), OutputWorkspace='input_workspace')
saved_calibration = load_calibration('input_workspace', 'BARSCAN', component='wing_detector',
                                     database='/HFIR/CG3/shared/tmp/calibration.json')
start_time = time.time()
saved_calibration.apply('input_workspace', output_workspace='output_workspace')
print('Applying the calibration took ', time.time() - start_time, 'seconds')

<h3>Visualizing the Effects of the Calibration</h3>
We plot intensities on the wing detector before and after the calibration. Notice we use argument <code>axes_mode='xy'</code> that instruct <code>plot_wing_detector</code> to plot intensities versus $X$ and $Y$ coordinates, instead of the default plotting. The default plotting is versus tube index and pixel index.

Notice: generating the plots versus $X$ and $Y$ will take about a minute

In [ ]:
plot_wing_detector('input_workspace', axes_mode='xy')  # before calibration
plot_wing_detector('output_workspace', axes_mode='xy')  # after calibration

If the calibration has worked, the bar should appear more **levelled** after the calibration.

Also notice that all values of $X$ are negative, as the wing detector is standing on the negative side of the X-axis

<h3>Viewing the Calibrated Positions and Heights</h3>
We can retrieve the pixel positions and heights from the calibrated <code>output_workspace</code>. The cell below will take for each pixel its vertical position and store this value as the intensity value in workspace <code>views.positions</code>. Later we can color plot this workspace to view the assigned positions to each pixel. The same process is done for pixel heights.

Again, extracting the ~50K positions and heights takes time.

In [ ]:
views = as_intensities('output_workspace', component='wing_detector')
plot_wing_detector(views.positions)
plot_wing_detector(views.heights)